# LangChain Use-Case Example: Processing Data

This example shows how you can use LangChain to feed data to an LLM to make decisions, classifications, extract text, or any of the other things language models can do with data.

You can run this code if you have an OpenAI API key.  If you have other API keys then you can run some of the demos that use multiple AI APIs from the same code.

## Setup code

The first code cell installs the LangChain modules and retrieves AI API credentials.  Please set up Colab secrets for any of these:

* `OPENAI_API_KEY`    -- To use the OpenAI API
* `AWS_ACCESS_KEY_ID` -- To use AWS Bedrock
* `AWS_SECRET_ACCESS_KEY`
* `AWS_REGION_NAME`

In [1]:
# Load secrets
import os
from dotenv import load_dotenv
load_dotenv()

def load_environment_variables(variable_names):
    for var_name in variable_names:
        if var_name not in os.environ:
            try:
                from google.colab import userdata
                value = userdata.get(var_name)
                if value:
                    os.environ[var_name] = value
            except ImportError:
                pass
        if var_name not in os.environ:
            raise ValueError(f"{var_name} not found. Please set it in .env file or Google Colab secrets.")
        else:
            print(f"Successfully loaded {var_name} from environment variables.")

variables_to_load = [
    "OPENAI_API_KEY",
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_REGION_NAME"
]

load_environment_variables(variables_to_load)

Successfully loaded OPENAI_API_KEY from environment variables.
Successfully loaded AWS_ACCESS_KEY_ID from environment variables.
Successfully loaded AWS_SECRET_ACCESS_KEY from environment variables.
Successfully loaded AWS_REGION_NAME from environment variables.


In [2]:
# Install necessary libraries
!pip install langchain langchain-community langchain-core langchain-openai

# LangChain setup: Show more about what's happening as it happens.
# from langchain.globals import set_debug
# set_debug(True)

## Example: Detect Something In A Dataset

This [HuggingFace dataset](https://huggingface.co/datasets/AyoubChLin/CNN_News_Articles_2011-2022) contains news articles from CNN.  What if we want to use an LLM to scan the articles, looking for something?  LangChain makes that pretty easy.

In [18]:
from datasets import load_dataset
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from tqdm import tqdm
import random

dataset = load_dataset("AyoubChLin/CNN_News_Articles_2011-2022", split="train")
sampled_data = random.sample(list(dataset), k=int(len(dataset) * 0.05))

model = ChatOpenAI(model_name="gpt-4o-mini")
prompt = PromptTemplate.from_template(
  "Does the following news article discuss space exploration missions? Answer with 'Yes' or 'No':\n\n{text}"
)
chain = prompt | model

for item in tqdm(sampled_data, desc="Processing articles", unit="article"):
  response = chain.invoke({"text": item["text"]})
  if response.content.strip().lower() == "yes":
    print(f"Text discusses space flight:\n{item['text'][:1000]}...")

Processing articles:   2%|▏         | 34/1610 [00:15<11:03,  2.38article/s]

Text discusses space flight:
Sign up for CNN's Wonder Theory science newsletter. Explore the universe with news on fascinating discoveries, scientific advancements and more. (CNN)Space travel is known to be notoriously rough on the human body, but new research has revealed just how hard it hits red blood cells. When we're on Earth, our bodies create and destroy 2 million of these cells per second. In space, astronauts experienced 3 million red blood cells destroyed per second, resulting in a loss of 54% more cells than people on Earth experience, according to a new study.Lower red blood cell counts in astronauts is known as space anemia."Space anemia has consistently been reported when astronauts returned to Earth since the first space missions, but we didn't know why," said study author Dr. Guy Trudel, a rehabilitation physician and researcher at The Ottawa Hospital and professor at the University of Ottawa, in a statement. Canadian astronaut David Saint-Jacques collected breath, ambi

Processing articles:   3%|▎         | 53/1610 [00:22<09:21,  2.77article/s]

Text discusses space flight:
Story highlightsPhobos-Grunt was supposed to land on one of the moons of MarsIt launched on November 9 but never escaped Earth orbitThe failed Russian space probe Phobos-Grunt fell back to Earth on Sunday, crashing in the southeastern Pacific Ocean off Chile, Russia's state news agency Itar-Tass announced.The spacecraft crashed about 1,250 km (775 miles) west of Chile's Wellington Island, according to a Russian military spokesman quoted by Itar-Tass.The nearly 15-ton probe was supposed to have landed on one of the moons of Mars and send back soil and rock samples. But its engines failed to boost it out of Earth orbit after its November 9 launch, and Russian officials estimated that fragments weighing up to 440 pounds could have survived re-entry. ...


Processing articles:  40%|███▉      | 637/1610 [04:42<08:40,  1.87article/s]

...

Processing articles:  99%|█████████▊| 1589/1610 [11:42<00:09,  2.33article/s]

Text discusses space flight:
Story highlights The Soyuz capsule docked to the space station late Tuesday Two veteran space travelers and one rookie is inside the Russian rocketThey'll spend the next six months in space The latest ISS crew set off hours earlier from KazakhstanA Russian rocket docked late Tuesday with the International Space Station, bringing two space veterans and a rookie astronaut one step closer to a long stay orbiting the Earth.The three -- Russian cosmonaut Fyodor Yurchikhin, American astronaut Karen Nyberg and Italian Luca Parmitano of the European Space Agency -- will spend the next six months at the space station until their scheduled departure in mid-November.Read more: Crew takes first fast-track flight to International Space StationThe Soyuz capsule carrying them into orbit lifted off at 2:31 a.m. Wednesday (4:31 p.m. ET Tuesday) from the Baikonur Cosmodrome in Kazakhstan for a nearly six-hour flight to the station. Once inside the space station, they will jo

Processing articles:  99%|█████████▉| 1597/1610 [11:45<00:04,  2.66article/s]

Text discusses space flight:
Story highlightsGene Seymour: Gene Roddenberry may have created "Star Trek," but Leonard Nimoy and character of Spock are inseparableHe says Nimoy had many other artistic endeavors, photography, directing, poetry, but he was, in the end, SpockGene Seymour is a film critic who has written about music, movies and culture for The New York Times, Newsday, Entertainment Weekly and The Washington Post. The opinions expressed in this commentary are solely those of the writer. (CNN)Everybody on the planet knows that Gene Roddenberry created Mr. Spock, the laconic, imperturbable extra-terrestrial First Officer for the Starship Enterprise. But Mr. Spock doesn't belong to Roddenberry, even though he is the grand exalted progenitor of everything that was, is, and forever will be "Star Trek."Mr. Spock belongs to Leonard Nimoy, who died Friday at age 83. And though he doesn't take Spock with him, he and Spock remain inseparable. Zachary Quinto, who plays Spock in the re-

Processing articles: 100%|██████████| 1610/1610 [11:50<00:00,  2.27article/s]
